# Aggregating

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/dc-wikia-data-clean.csv')

## Basic summarizing and descriptive statistics

In [3]:
df.mean()

page_id        147441.209252
appearances        23.625134
year             1989.766662
dtype: float64

In [4]:
df.std()

page_id        108388.631149
appearances        87.378509
year               16.824194
dtype: float64

In [5]:
df['sex'].unique()

array(['Male', 'Female', nan, 'Genderless', 'Transgender'], dtype=object)

In [6]:
df['sex'].value_counts()

Male           4783
Female         1967
Genderless       20
Transgender       1
Name: sex, dtype: int64

In [7]:
df['year'].min()

1935.0

In [8]:
df['year'].max()

2013.0

## `groupby`

**Figure copied from [Jake Vanderplas's book](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.08-Aggregation-and-Grouping.ipynb).**

![title](figures/jake-vanderplas-split-apply-combine.png)

### Basic built-in aggregation functions

`count`, `sum`, `mean`, `median`, `std`, `var`, `min`, `max`, `prod`, `first`, `last`.

In [11]:
all(df['page_id'].notnull())

True

In [12]:
df.groupby('sex').count()['page_id']

sex
Female         1967
Genderless       20
Male           4783
Transgender       1
Name: page_id, dtype: int64

In [13]:
df.groupby('sex').mean()

,page_id,appearances,year
sex,,,
Female,159307.531774,22.484574,1992.621983
Genderless,132061.750000,12.842105,1990.850000
Male,141814.427138,24.499890,1988.532841
Transgender,317067.000000,4.000000,2009.000000


With multiple level (search for `MultiIndex` for more info).

In [15]:
df[df['sex'] == 'Transgender']

,page_id,name,urlslug,id,align,eye,hair,sex,gsm,alive,appearances,first appearance,year
3877,317067,Daystar (New Earth),\/wiki\/Daystar_(New_Earth),NaN,Bad,NaN,NaN,Transgender,NaN,Deceased,4.0,"2009, October",2009.0


In [14]:
df.groupby(['sex', 'align']).count()

page_id  name  urlslug    id  eye  hair  gsm  alive  \
sex         align                                                           
Female      Bad           597   597      597   419  325   472    7    596   
            Good          953   953      953   714  582   812   18    953   
            Neutral       196   196      196   138  125   169    3    196   
            Reformed        1     1        1     0    1     1    0      1   
Genderless  Bad            11    11       11     9    3     1    0     11   
            Good            6     6        6     4    2     2    0      6   
            Neutral         3     3        3     3    3     0    1      3   
Male        Bad          2223  2223     2223  1542  861  1204    8   2223   
            Good         1843  1843     1843  1419  919  1306   25   1842   
            Neutral       359   359      359   254  191   253    1    359   
            Reformed        2     2        2     2    1     2    0      2   
Transgender Bad             1     1        1     0    0     0    0      1   

                      appearances  first appearance  year  
sex         align                                          
Female      Bad               568               596   596  
            Good              909               941   941  
            Neutral           193               194   194  
            Reformed            1                 1     1  
Genderless  Bad                10                11    11  
            Good                6                 6     6  
            Neutral             3                 3     3  
Male        Bad              2088              2208  2208  
            Good             1756              1819  1819  
            Neutral           348               353   353  
            Reformed            2                 2     2  
Transgender Bad                 1                 1     1

### Custom aggregations

Specifying pandas built-in functions by name.

In [16]:
df.groupby('sex').agg({'page_id': 'count'})

,page_id
sex,
Female,1967
Genderless,20
Male,4783
Transgender,1


Using multiple functions for the same column.

In [17]:
df.groupby('sex').agg({'appearances': ['mean', 'std']})

appearances           
                   mean        std
sex                               
Female        22.484574  68.717080
Genderless    12.842105  11.922263
Male          24.499890  95.168200
Transgender    4.000000        NaN

Using custom python functions.

In [19]:
def values_range(x):
    return max(x) - min(x)

In [21]:
df.groupby('sex').agg({'appearances': values_range})

,appearances
sex,
Female,1230.0
Genderless,35.0
Male,3092.0
Transgender,0.0


## *Exercise*

Among bisexual characters, what is the sex that appears the most? Is that the same for homosexual characters?

In [33]:
(
    df.groupby(['gsm', 'sex'])
    .agg({'appearances': ['sum', 'mean'], 'page_id': 'count'})
#     .sort_values('appearances')
    .rename(columns={'page_id': 'count'})
)

appearances             count
                              sum        mean count
gsm        sex                                     
Bisexual   Female           322.0   64.400000     5
           Genderless        20.0   20.000000     1
           Male             436.0  109.000000     4
Homosexual Female          1135.0   47.291667    24
           Male            1001.0   33.366667    30